# Data Preprocessing

In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import scipy.sparse
import joblib
from collections import Counter
import re
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/brina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/brina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/brina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Parameters

In [1]:
# change this accordingly
project_path = "/home/brina/nus-mcomp/sem3/cs4248-natural-language-processing/Project/CS4248-NLP-Project"

save_path = f"{project_path}/scicite_preprocessed"

## 1. Load dataset

In [3]:
dataset_dir = f"{project_path}/scicite"
train_df = pd.read_json(f"{dataset_dir}/train.jsonl", lines=True)
test_df = pd.read_json(f"{dataset_dir}/test.jsonl", lines=True)

display(
    train_df.head(),
    train_df.shape,
    test_df.head(),
    test_df.shape
)

,source,citeEnd,sectionName,citeStart,string,label,label_confidence,citingPaperId,citedPaperId,isKeyCitation,id,unique_id,excerpt_index,label2,label2_confidence
0,explicit,175.0,Introduction,168.0,"However, how frataxin interacts with the Fe-S ...",background,1.0,1872080baa7d30ec8fb87be9a65358cd3a7fb649,894be9b4ea46a5c422e81ef3c241072d4c73fdc0,True,1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be...,1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be...,11,NaN,NaN
1,explicit,36.0,Novel Quantitative Trait Loci for Seminal Root...,16.0,"In the study by Hickey et al. (2012), spikes w...",background,1.0,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b,b6642e19efb8db5623b3cc4eef1c5822a6151107,True,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b>b6642...,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b>b6642...,2,NaN,NaN
2,explicit,228.0,Introduction,225.0,"The drug also reduces catecholamine secretion,...",background,1.0,9cdf605beb1aa1078f235c4332b3024daa8b31dc,4e6a17fb8d7a3cada601d942e22eb5da6d01adbd,False,9cdf605beb1aa1078f235c4332b3024daa8b31dc>4e6a1...,9cdf605beb1aa1078f235c4332b3024daa8b31dc>4e6a1...,0,NaN,NaN
3,explicit,110.0,Discussion,46.0,By clustering with lowly aggressive close kin ...,background,1.0,d9f3207db0c79a3b154f3875c9760cc6b056904b,2cc6ff899bf17666ad35893524a4d61624555ed7,False,d9f3207db0c79a3b154f3875c9760cc6b056904b>2cc6f...,d9f3207db0c79a3b154f3875c9760cc6b056904b>2cc6f...,3,NaN,NaN
4,explicit,239.0,Discussion,234.0,Ophthalmic symptoms are rare manifestations of...,background,1.0,88b86556857f4374842d2af2e359576806239175,a5bb0ff1a026944d2a47a155462959af2b8505a8,False,88b86556857f4374842d2af2e359576806239175>a5bb0...,88b86556857f4374842d2af2e359576806239175>a5bb0...,2,NaN,NaN


(8243, 15)

,source,citeEnd,sectionName,citeStart,string,label,citingPaperId,citedPaperId,isKeyCitation,id,unique_id,excerpt_index,label2,label_confidence
0,acronym,31.0,,28.0,"Chapel, as well as X10 [2], UPC [3] , CoArray ...",background,2c6797dab4c118cb73197f65ba39dacc99ac743d,95c37bc99982d33873fd141ee00857160fd717a0,True,2c6797dab4c118cb73197f65ba39dacc99ac743d>95c37...,2c6797dab4c118cb73197f65ba39dacc99ac743d>95c37...,6,NaN,NaN
1,explicit,166.0,Discussion,156.0,"In addition, the result of the present study s...",result,fa7145adc9f8cfb8af7a189d9040c13c84ced094,20e23b4f76761d246a7c3b00b80e139e2008f77d,False,fa7145adc9f8cfb8af7a189d9040c13c84ced094>20e23...,fa7145adc9f8cfb8af7a189d9040c13c84ced094>20e23...,0,supportive,NaN
2,explicit,145.0,Discussion,137.0,Several instruments that more specifically add...,background,98a8d8c0c5dae246720d4f339b88e8a9f44e3002,bd222c7ec83dadefba513738290b3624f6dd6b21,True,98a8d8c0c5dae246720d4f339b88e8a9f44e3002>bd222...,98a8d8c0c5dae246720d4f339b88e8a9f44e3002>bd222...,1,NaN,NaN
3,explicit,158.0,Methods,137.0,Organotypic hippocampal slice cultures\nInterf...,method,aeb178ef1910a61152cd74209c28641199c82855,754c04953c261072fa367f4104e3deff082d9484,False,aeb178ef1910a61152cd74209c28641199c82855>754c0...,aeb178ef1910a61152cd74209c28641199c82855>754c0...,1,NaN,1.0
4,explicit,186.0,RESULTS,178.0,Activated PBMC are the basis of the standard P...,background,e4d2591ac3bb65e2ec59f092884a7b15b8018592,f0fb468a54fe8021bc7986a1618222c4fcd16df4,False,e4d2591ac3bb65e2ec59f092884a7b15b8018592>f0fb4...,e4d2591ac3bb65e2ec59f092884a7b15b8018592>f0fb4...,6,NaN,NaN


(1861, 14)

## 2. Text Preprocessing

#### Clean 'string' column
- Remove citations (e.g., [12,22]).
- Convert text to lowercase.
- Remove punctuation.
- Remove numbers.
- Remove stopwords.
- Lemmatize words using NLTK.
- Replace multiple spaces with a single space.

In [4]:
# keep only 'string' and 'label' columns
train_df = train_df[["label", "string"]]
test_df = test_df[["label", "string"]]

In [5]:
# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to clean text
def clean_text(text):
    if pd.isna(text):  # Handle NaN values
        return ""
    
    # Remove various citation formats
    text = re.sub(r"\[[\d,\s–-]+\]", "", text)  # Remove citations like [1–5], [21, 23, 33, 64, 100, 101, 107–118]
    text = re.sub(r"\([\d,\s–-]+\)", "", text)  # Remove citations like (96-98), (7)
    text = re.sub(r"\b[A-Z][a-z]+ et al\.\s*\(\d{4}\)", "", text)  # Doe et al. (2021)
    text = re.sub(r"\(\w+,\s*\d{4}(?:;\s*\w+,\s*\d{4})*\)", "", text)  # (Smith, 2020; Johnson, 2019)
    text = re.sub(r"\b(ICML|NeurIPS|CVPR|JMLR|arXiv|vol\.\s*\d+)\b", "", text)  # Journal/conference names

    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = re.sub(r"\d+", "", text)  # Remove numbers
    
    tokens = word_tokenize(text)  # Tokenize text
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Lemmatization & remove stopwords
    
    return " ".join(tokens)  # Reconstruct sentence

# Apply text cleaning function
train_df['string'] = train_df['string'].apply(clean_text)
test_df['string'] = test_df['string'].apply(clean_text)

# Display results
display(
    train_df.head(),
    test_df.head()
)

,label,string
0,background,however frataxin interacts fe cluster biosynth...
1,background,study hickey et al spike sampled field point p...
2,background,drug also reduces catecholamine secretion ther...
3,background,clustering lowly aggressive close kin king ab ...
4,background,ophthalmic symptom rare manifestation intracra...


,label,string
0,background,chapel well x upc coarray fortran titanium rel...
1,result,addition result present study support previous...
2,background,several instrument specifically address patien...
3,method,organotypic hippocampal slice culture interfac...
4,background,activated pbmc basis standard pbmc blast assay...


- TF-IDF Vectorization of 'string' column

In [6]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3)) # captures unigrams, bigrams, and trigrams
X_train_tfidf = vectorizer.fit_transform(train_df["string"])

# Get the original number of features
original_num_features = len(vectorizer.get_feature_names_out())
print(f"Original number of features: {original_num_features}")

Original number of features: 269419


In [7]:
# Initialize the vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=5000)

# Fit and transform the 'string' column in the training set
X_train_tfidf = vectorizer.fit_transform(train_df["string"])

# Transform the 'string' column in the test set
X_test_tfidf = vectorizer.transform(test_df["string"])

# Convert the sparse matrix to a DataFrame for training set
tfidf_train_df = pd.DataFrame(X_train_tfidf.toarray(), columns=vectorizer.get_feature_names_out(), index=train_df.index)

# Rename columns of the TF-IDF DataFrame to avoid overlap (add '_tfidf' suffix)
tfidf_train_df.columns = [col + '_tfidf' for col in tfidf_train_df.columns]

# Drop the original 'string' column from the train DataFrame
train_df.drop(columns=['string'], inplace=True)

# Concatenate the original DataFrame with the TF-IDF DataFrame
train_df = pd.concat([train_df, tfidf_train_df], axis=1)

# Repeat the process for the test DataFrame
tfidf_test_df = pd.DataFrame(X_test_tfidf.toarray(), columns=vectorizer.get_feature_names_out(), index=test_df.index)
tfidf_test_df.columns = [col + '_tfidf' for col in tfidf_test_df.columns]

# Drop the original 'string' column from the test DataFrame
test_df.drop(columns=['string'], inplace=True)

# Concatenate the original test DataFrame with the TF-IDF DataFrame
test_df = pd.concat([test_df, tfidf_test_df], axis=1)

# Display the updated train and test DataFrames
display(
    train_df.head(),
    test_df.head()
)

,label,aa_tfidf,ab_tfidf,abdominal_tfidf,abdullah_tfidf,abeler_tfidf,abeler et_tfidf,abeler et al_tfidf,abi_tfidf,ability_tfidf,...,zhou et_tfidf,zhou et al_tfidf,zhu_tfidf,zhu et_tfidf,zhu et al_tfidf,zinc_tfidf,zn_tfidf,zone_tfidf,äì_tfidf,μm_tfidf
0,background,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,background,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,background,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,background,0.0,0.152298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,background,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,label,aa_tfidf,ab_tfidf,abdominal_tfidf,abdullah_tfidf,abeler_tfidf,abeler et_tfidf,abeler et al_tfidf,abi_tfidf,ability_tfidf,...,zhou et_tfidf,zhou et al_tfidf,zhu_tfidf,zhu et_tfidf,zhu et al_tfidf,zinc_tfidf,zn_tfidf,zone_tfidf,äì_tfidf,μm_tfidf
0,background,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,result,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,background,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,method,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,background,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 2.6 Label encoding for 'label' column

In [10]:
# Initialize Label Encoder
label_encoder = LabelEncoder()

# Encode labels directly in the 'label' column
train_df["label"] = label_encoder.fit_transform(train_df["label"])
test_df["label"] = label_encoder.fit_transform(test_df["label"])
joblib.dump(label_encoder, 'scicite_preprocessed/label_encoder.pkl')

# Show label mappings
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Labels:", label_mapping)

Labels: {'background': 0, 'method': 1, 'result': 2}


## Save processed dataset

In [11]:
train_df.to_csv(f"{save_path}/train-string-only.csv", index=False)
test_df.to_csv(f"{save_path}/test-string-only.csv", index=False)

# TODO: ML pipeline

- [x] Clean data
- [ ] Upsampling
- [x] Word vectorization
- [ ] Word embedding
- [ ] PCA 
- [ ] top-k features selection
- [ ] Training the classfication model